# Datenanalyse Seattle Library

In [466]:
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import csv

## (1) Titel und Autor für meistausgeliehene Medien im Januar 2023

### Daten runterladen

Daten werden schon bei Abfrage bei der API gefiltert nach digitalen und physischen Medien, Entleihjahr und -monat und Mindestanzahl von Ausleihen. Die Mindestanzahl wurde durch Testen so ermittelt, dass die Ausgabe unter den maximal 10.000 Einheiten blieb.
Die Query wurde encoded mithilfe von [https://www.coderstool.com/querystring-encode](https://www.coderstool.com/querystring-encode)

In [467]:
# Query: Ausleihen im Januar 2023 von physischen Medien mit Ausleihzahl von mindestens 6
url = f"https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=usageclass='Physical'&checkoutyear=2023&checkoutmonth=01%20AND%20checkouts%3E6&$limit=10000"
local_file_phys = "physical_2023-01.csv"
urllib.request.urlretrieve(url, local_file_phys)

('physical_2023-01.csv', <http.client.HTTPMessage at 0x7983d59ab770>)

In [468]:
# Query: Ausleihen im Januar 2023 von digitalen Medien mit Ausleihzahl von mindestens 8
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=usageclass='Digital'&checkoutyear=2023&checkoutmonth=01%20AND%20checkouts%3E8&$limit=10000"
local_file_dig = "digital_2023-01.csv"
urllib.request.urlretrieve(url, local_file_dig)

('digital_2023-01.csv', <http.client.HTTPMessage at 0x7983d5189550>)

In [469]:
# Laden in Dateframes
df_physical = pd.read_csv(local_file_phys)
df_digital = pd.read_csv(local_file_dig)

In [470]:
df_physical.shape

(9975, 12)

In [471]:
df_digital.shape

(8867, 12)

### Die beiden meistausgeliehenen Objekte (physisch und digital)

In [472]:
max_phys_int = df_physical["checkouts"].max()
max_dig_int = df_digital["checkouts"].max()

In [473]:
print("Maximale Ausleihzahl eines physischen Objekts: " + str(max_phys_int))
print("Maximale Ausleihzahl eines digitalen Objekts: " + str(max_dig_int))

Maximale Ausleihzahl eines physischen Objekts: 2115
Maximale Ausleihzahl eines digitalen Objekts: 791


In [474]:
max_phys_row = df_physical.loc[df_physical["checkouts"] == max_phys_int]
max_dig_row = df_digital.loc[df_digital["checkouts"] == max_dig_int]

In [475]:
max_phys_row

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
1123,Physical,Horizon,MIXED,2023,1,2115,Headphones / Seattle Public Library.,NaN,NaN,NaN,Seattle Public Library,NaN


In [476]:
max_dig_row

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
5137,Digital,OverDrive,AUDIOBOOK,2023,1,791,101 Essays That Will Change the Way You Think ...,9.781977e+12,Brianna Wiest,"Essays, Philosophy, Self-Improvement, Nonfiction","Tantor Media, Inc",2021


Kombination beider Zeilen in eine Tabelle (hat keinen tieferen Sinn, ist nur zum concat-Ausprobieren)

In [477]:
type(max_dig_row)

pandas.core.frame.DataFrame

In [478]:
max_df_combined = pd.concat([max_phys_row, max_dig_row])
max_df_combined

/tmp/ipykernel_47231/331941362.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  max_df_combined = pd.concat([max_phys_row, max_dig_row])


,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
1123,Physical,Horizon,MIXED,2023,1,2115,Headphones / Seattle Public Library.,NaN,NaN,NaN,Seattle Public Library,NaN
5137,Digital,OverDrive,AUDIOBOOK,2023,1,791,101 Essays That Will Change the Way You Think ...,9.781977e+12,Brianna Wiest,"Essays, Philosophy, Self-Improvement, Nonfiction","Tantor Media, Inc",2021


### Die zehn meistausgeliehenen physischen Objekte

In [479]:
ranking_checkouts_phys_series = df_physical["checkouts"].nlargest(10)

In [480]:
type(ranking_checkouts_phys_series)

pandas.core.series.Series

In [481]:
print(ranking_checkouts_phys_series)

1123    2115
4770     841
8074     485
2481     432
5942     426
4870     416
9268     416
9699     381
301      377
1176     373
Name: checkouts, dtype: int64


In [482]:
df_subset_most_checkouts_phys = df_physical.nlargest(10,"checkouts")
df_subset_most_checkouts_dig = df_digital.nlargest(10,"checkouts")

In [483]:
type(df_subset_most_checkouts_dig)

pandas.core.frame.DataFrame

### Die zehn meistgeliehenen physischen Objekte im Januar 2023 mit Art des Objekts, Titel, Urheber und Anzahl der Ausleihen

In [484]:
df_subset_most_checkouts_phys[["materialtype", "title", "creator", "checkouts"]]

,materialtype,title,creator,checkouts
1123,MIXED,Headphones / Seattle Public Library.,NaN,2115
4770,MIXED,SPL HotSpot : connecting Seattle / [distribute...,NaN,841
8074,BOOK,Spare / Prince Harry.,"Harry, Prince, Duke of Sussex, 1984-",485
2481,BOOK,Lessons in chemistry / Bonnie Garmus.,"Garmus, Bonnie",432
5942,BOOK,Our missing hearts : a novel / Celeste Ng.,"Ng, Celeste",426
4870,BOOK,I'm glad my mom died / Jennette McCurdy.,"McCurdy, Jennette, 1992-",416
9268,BOOK,The light we carry : overcoming in uncertain t...,"Obama, Michelle, 1964-",416
9699,BOOK,Dinner in one : exceptional & easy one-pan mea...,"Clark, Melissa",381
301,BOOK,Foster / Claire Keegan.,"Keegan, Claire",377
1176,BOOK,The passenger / Cormac McCarthy.,"McCarthy, Cormac, 1933-",373


### Die zehn meistgeliehenen digitalen Objekte im Januar 2023 mit Art des Objekts, Titel, Urheber und Anzahl der Ausleihen

In [485]:
df_subset_most_checkouts_dig[["materialtype", "title", "creator", "checkouts"]]

,materialtype,title,creator,checkouts
5137,AUDIOBOOK,101 Essays That Will Change the Way You Think ...,Brianna Wiest,791
441,AUDIOBOOK,"Braiding Sweetgrass: Indigenous Wisdom, Scient...",Robin Wall Kimmerer,729
1426,EBOOK,Lessons in Chemistry: A Novel,Bonnie Garmus,681
660,EBOOK,"Tomorrow, and Tomorrow, and Tomorrow: A novel",Gabrielle Zevin,667
5570,AUDIOBOOK,I'm Glad My Mom Died (unabridged),Jennette McCurdy,564
3172,AUDIOBOOK,Spare (unabridged),"Prince Harry, The Duke of Sussex",533
8709,AUDIOBOOK,A Court of Thorns and Roses: A Court of Thorns...,Sarah J. Maas,515
4457,EBOOK,Cloud Cuckoo Land: A Novel,Anthony Doerr,478
8171,AUDIOBOOK,"Tomorrow, and Tomorrow, and Tomorrow: A novel ...",Gabrielle Zevin,466
6541,EBOOK,The Last Thing He Told Me: A Novel,Laura Dave,440


## (2) Wie viele Comics mit Erscheinungsjahr 2005 finden sich in der Collection? 

### ILS Data Dictionary laden

In [486]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json"
local_file = "seattle_dd.json"
urllib.request.urlretrieve(url, local_file)

('seattle_dd.json', <http.client.HTTPMessage at 0x7983d5973380>)

In [487]:
df_data_dict = pd.read_json("seattle_dd.json")

In [488]:
df_data_dict

,code,description,code_type,format_group,format_subgroup,category_group,age_group,category_subgroup
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,Adult,NaN
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,Adult,NaN
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,Adult,NaN
3,napass,NA-Discovery Pass,ItemCollection,Media,Pass,Miscellaneous,Adult,NaN
4,ccbocd,CC - Children's Books on CD,ItemCollection,Media,Audiobook Disc,Fiction,Juvenile,NaN
...,...,...,...,...,...,...,...,...
620,acblu,Blu-ray: Adult/YA,ItemType,Media,Video Disc,NaN,Adult,NaN
621,ncblu,NC-Children's Blu-rays,ItemCollection,Media,Video Disc,Fiction,Juvenile,NaN
622,cazinec,ZAPP zine collection circulating,ItemCollection,Print,Book,Miscellaneous,Adult,NaN
623,cavalf,CA0-Valuable Material Folio,ItemCollection,NaN,NaN,NaN,NaN,NaN


In [489]:
df_data_dict["code_type"].unique()

array(['ItemCollection', 'ItemType', 'Location', 'ItemTypeDetail'],
      dtype=object)

Preparing data dict for merge with Library Inventory Collection:
- Filtering for only code_type ItemCollection, dropping that column and renaming code column as in the lis table (itemcollection)

In [518]:
df_data_dict_only_itemcoll = df_data_dict[df_data_dict["code_type"]=="ItemCollection"]
df_data_dict_only_itemcoll = df_data_dict_only_itemcoll.drop(columns=["code_type"])
df_data_dict_only_itemcoll = df_data_dict_only_itemcoll.rename(columns={"code":"itemcollection"})
df_data_dict_only_itemcoll

,itemcollection,description,format_group,format_subgroup,category_group,age_group,category_subgroup
0,cazover,CA7-zine collection oversize,Print,Book,Periodical,Adult,NaN
1,caziner,CA7-zine collection reference,Print,Book,Periodical,Adult,NaN
2,cazval,CA7-zine collection valuable mat.,Print,Book,Periodical,Adult,NaN
3,napass,NA-Discovery Pass,Media,Pass,Miscellaneous,Adult,NaN
4,ccbocd,CC - Children's Books on CD,Media,Audiobook Disc,Fiction,Juvenile,NaN
...,...,...,...,...,...,...,...
618,nablu,"NA-Blu-ray, Fiction",Media,Video Disc,Fiction,Adult,NaN
619,ccblu,"CC-Children's Blu-ray, Fiction",Media,Video Disc,Fiction,Juvenile,NaN
621,ncblu,NC-Children's Blu-rays,Media,Video Disc,Fiction,Juvenile,NaN
622,cazinec,ZAPP zine collection circulating,Print,Book,Miscellaneous,Adult,NaN


In [514]:
df_data_dict_only_itemcoll["category_subgroup"].unique()

array([nan, 'Biography', 'Peak Picks', 'Comic', 'Large Print', 'Holiday',
       'Picture', 'ESL'], dtype=object)

### Load Library Collection Inventory

Versuch: Titel mit ERscheinungsjahr 2005 herunterzuladen, allerdings: in Feld publicationyear verschiedene Formatierungen für Druck/Erscheinungs/Copyrightjahr, deshalb ggf. nicht alles gefunden.
https://data.seattle.gov/api/views/6vkj-f5xf/files/61a7279a-85fb-4061-8a65-6c09bb63ecbe?download=true&filename=Library%20Collection%20Inventory%20FAQs.pdf 

Herunterladen sollte eigentlich schrittweise erfolgen - aber wenn "order" in Query mit angegeben, hat das Herunterladen nicht funktioniert (jedenfalls nicht nach mehreren Minuten warten). 

In [494]:
limit = 10000
offset = 0
base_url = "https://data.seattle.gov/resource/6vkj-f5xf.json"
query = "?$where=publicationyear%20in%20(%272005%27,%20%27[2005]%27,%20%272005.%27,%27[2005].%27,%27[2005?]%27,%27[c2005.]%27,%27[p2005.]%27)"
#order = "&$order=bibnum"
order = ""
df_seattle_lci2005 = pd.DataFrame()
df_temp = pd.DataFrame()
#url = "https://data.seattle.gov/resource/6vkj-f5xf.json?$where=publicationyear%20in%20(%272005%27,%20%27[2005]%27)&$limit=20000"
#url = "https://data.seattle.gov/resource/6vkj-f5xf.json?&$limit=10000"
found_data = True
n = 0
while found_data and n < 3:
    local_file = f"seattle-lci_{n}.json"
    limit_and_offset = f"&$limit={limit}&$offset={offset}"
    print(base_url+query+limit_and_offset+order)
    urllib.request.urlretrieve(base_url+query+limit_and_offset, local_file)
    offset +=10000
    n+=1
    df_temp = pd.read_json(local_file)
    df_seattle_lci2005 = pd.concat([df_seattle_lci2005, df_temp])
    if df_temp.empty:
        print("leeres Datenset")
        found_data = False

https://data.seattle.gov/resource/6vkj-f5xf.json?$where=publicationyear%20in%20(%272005%27,%20%27[2005]%27,%20%272005.%27,%27[2005].%27,%27[2005?]%27,%27[c2005.]%27,%27[p2005.]%27)&$limit=10000&$offset=0
https://data.seattle.gov/resource/6vkj-f5xf.json?$where=publicationyear%20in%20(%272005%27,%20%27[2005]%27,%20%272005.%27,%27[2005].%27,%27[2005?]%27,%27[c2005.]%27,%27[p2005.]%27)&$limit=10000&$offset=10000
https://data.seattle.gov/resource/6vkj-f5xf.json?$where=publicationyear%20in%20(%272005%27,%20%27[2005]%27,%20%272005.%27,%27[2005].%27,%27[2005?]%27,%27[c2005.]%27,%27[p2005.]%27)&$limit=10000&$offset=20000


In [498]:
df_seattle_lci2005 = df_seattle_lci2005.drop_duplicates()

In [503]:
df_seattle_lci2005 = df_seattle_lci2005.reset_index()

In [504]:
df_seattle_lci2005["publicationyear"].unique()

array(['2005', '[2005?]', '[2005]'], dtype=object)

In [521]:
df_seattle_lci2005[pd.notnull(df_seattle_lci2005["itemcollection"])]

,index,bibnum,title,author,isbn,publicationyear,publisher,subjects,itemtype,itemcollection,floatingitem,itemlocation,reportdate,itemcount
0,0,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",2005,"HY International Printers,",Amharic fiction,acbk,naln,NA,rbe,2018-09-01T00:00:00.000,1
1,1,3276207,Seattle streetcar network south Jackson Street...,NaN,NaN,2005,"[Dept. of Transportation?],",Electric railroads Cars Washington State Seatt...,arbk,caseapm,NA,cen,2018-02-01T00:00:00.000,1
2,2,2874721,Nate the Great and the Halloween hunt / by Mar...,"Sharmat, Marjorie Weinman","0440403413, 0758706979, 0833562320, 1439548978...",2005,"Yearling Book,",Nate the Great Fictitious character Juvenile f...,jcbk,nchol,NA,rbe,2018-12-01T00:00:00.000,1
3,3,3409679,The perils of peppermints / Barbara Brooks Wal...,"Wallace, Barbara Brooks, 1922-","068985045X, 9780689850455",2005,"Aladdin Paperbacks,","Swindlers and swindling Juvenile fiction, Inhe...",jcbk,ncfic,NA,mag,2018-12-01T00:00:00.000,1
4,4,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",2005,"HY International Printers,",Amharic fiction,acbk,naln,NA,rbe,2018-07-01T00:00:00.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,9995,2295671,Lord Byron's novel : the evening land / John C...,"Crowley, John, 1942-",0060556587,[2005],"William Morrow,",Byron George Gordon Byron Baron 1788 1824 Fict...,acbk,nafic,NA,bal,2018-02-01T00:00:00.000,1
29993,9996,2800925,The Bitch Posse / Martha O'Connor.,"O'Connor, Martha","0312333935, 9780312333935",[2005],"St. Martin's Griffin,","Women authors Fiction, Married women Fiction, ...",acbk,nafic,NA,lcy,2018-09-01T00:00:00.000,1
29994,9997,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",[2005],"Beginner Books,","Camping Juvenile fiction, Individuality Juveni...",jcbk,ncrdr,Floating,nga,2018-07-01T00:00:00.000,1
29995,9998,2284791,Environmental restoration program final 2005 g...,ERM-West (Firm),NaN,[2005],"[The Firm],",Washington State Air National Guard Combat Com...,arbk,casea,NA,cen,2018-11-01T00:00:00.000,1


In [522]:
lci_table_reduced = df_seattle_lci2005[["bibnum", "title", "author", "isbn", "itemcollection"]]

In [523]:
lci_table_reduced

,bibnum,title,author,isbn,itemcollection
0,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",naln
1,3276207,Seattle streetcar network south Jackson Street...,NaN,NaN,caseapm
2,2874721,Nate the Great and the Halloween hunt / by Mar...,"Sharmat, Marjorie Weinman","0440403413, 0758706979, 0833562320, 1439548978...",nchol
3,3409679,The perils of peppermints / Barbara Brooks Wal...,"Wallace, Barbara Brooks, 1922-","068985045X, 9780689850455",ncfic
4,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",naln
...,...,...,...,...,...
29992,2295671,Lord Byron's novel : the evening land / John C...,"Crowley, John, 1942-",0060556587,nafic
29993,2800925,The Bitch Posse / Martha O'Connor.,"O'Connor, Martha","0312333935, 9780312333935",nafic
29994,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ncrdr
29995,2284791,Environmental restoration program final 2005 g...,ERM-West (Firm),NaN,casea


In [528]:
lci_tables_combined = pd.merge(lci_table_reduced, df_data_dict_only_itemcoll, how="left", on="itemcollection")

In [529]:
lci_tables_combined

,bibnum,title,author,isbn,itemcollection,description,format_group,format_subgroup,category_group,age_group,category_subgroup
0,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",naln,NA-Languages Collection,Print,Book,Language,Adult,NaN
1,3276207,Seattle streetcar network south Jackson Street...,NaN,NaN,caseapm,CA0-Seattle Room Pamphlet File,Print,Book,Miscellaneous,Adult,NaN
2,2874721,Nate the Great and the Halloween hunt / by Mar...,"Sharmat, Marjorie Weinman","0440403413, 0758706979, 0833562320, 1439548978...",nchol,NC-Children's Holiday,Print,Book,Fiction,Juvenile,Holiday
3,3409679,The perils of peppermints / Barbara Brooks Wal...,"Wallace, Barbara Brooks, 1922-","068985045X, 9780689850455",ncfic,NC-Children's Fiction,Print,Book,Fiction,Juvenile,NaN
4,3030549,Dinā : lebolad / Derebu ʾAdarā.,Derebu ʾAdarā,"9789994498604, 9994498606",naln,NA-Languages Collection,Print,Book,Language,Adult,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29992,2295671,Lord Byron's novel : the evening land / John C...,"Crowley, John, 1942-",0060556587,nafic,NA-Fiction,Print,Book,Fiction,Adult,NaN
29993,2800925,The Bitch Posse / Martha O'Connor.,"O'Connor, Martha","0312333935, 9780312333935",nafic,NA-Fiction,Print,Book,Fiction,Adult,NaN
29994,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ncrdr,NC--Children's Readers,Print,Book,Fiction,Juvenile,Picture
29995,2284791,Environmental restoration program final 2005 g...,ERM-West (Firm),NaN,casea,CA0-Seattle Coll.,Print,Book,Miscellaneous,Adult,NaN


### Comics in der heruntergeladenen Auswahl an Titeln

In [530]:
lci_tables_combined[lci_tables_combined["category_subgroup"]=="Comic"]

,bibnum,title,author,isbn,itemcollection,description,format_group,format_subgroup,category_group,age_group,category_subgroup
20734,2293013,"Fables. [5], The mean seasons / fables created...","Willingham, Bill",1401204864,nacomic,NA-Comics/Graphic Novels,Print,Book,Fiction,Adult,Comic
20740,2356219,"Bone. [Vol. 1], Out from Boneville / by Jeff S...","Smith, Jeff, 1960 February 27-","0439706238, 0439706408, 1415578508, 9780439706...",nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
20785,2356219,"Bone. [Vol. 1], Out from Boneville / by Jeff S...","Smith, Jeff, 1960 February 27-","0439706238, 0439706408, 1415578508, 9780439706...",nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
20803,2356219,"Bone. [Vol. 1], Out from Boneville / by Jeff S...","Smith, Jeff, 1960 February 27-","0439706238, 0439706408, 1415578508, 9780439706...",nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
20822,3009465,Chocolat. Vol. 1 / [story and art by] JiSang S...,"Shin, JiSang","8952744535, 9788952744531",nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
...,...,...,...,...,...,...,...,...,...,...,...
29768,2336672,"Catwoman : when in Rome / Jeph Loeb, writer ; ...","Loeb, Jeph",1401204325,nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
29780,2930607,Hana-Kimi : For you in full blossom. 8 / story...,"Nakajo, Hisaya","1421500078, 9781421500072",cycomic,CY3-Teen Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
29802,2406944,"Batman : year one / Frank Miller, writer ; Dav...","Miller, Frank, 1957-","1401206905, 1401207529, 9781401206901, 9781401...",nycomic,NY-Teen - Comics & Graphic Novels,Print,Book,Fiction,Teen,Comic
29854,2325759,"Ex machina. Book 2, Tag / Brian K. Vaughan, wr...","Vaughan, Brian K.",1401206263,nacomic,NA-Comics/Graphic Novels,Print,Book,Fiction,Adult,Comic


Ausleihen physischer Objekte im Januar 2023 mit reduzierter Spaltenauswahl

In [558]:
df_physical_reduced = df_physical[["materialtype","checkouts","title","creator","isbn","subjects"]]

In [559]:
df_physical_reduced = df_physical_reduced.rename(columns= {"creator":"author"})

In [560]:
df_checkout_lci_dd_combined = pd.merge(lci_tables_combined, df_physical_reduced, how="inner", left_on=["author","title"], right_on=["author","title"])

In [561]:
df_checkout_lci_dd_combined

,bibnum,title,author,isbn_x,itemcollection,description,format_group,format_subgroup,category_group,age_group,category_subgroup,materialtype,checkouts,isbn_y,subjects
0,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ccrdr,CC-Children's Readers,Print,Book,Fiction,Juvenile,Picture,BOOK,21,"0375829652, 0375929657","Camping Juvenile fiction, Individuality Juveni..."
1,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ncrdr,NC--Children's Readers,Print,Book,Fiction,Juvenile,Picture,BOOK,21,"0375829652, 0375929657","Camping Juvenile fiction, Individuality Juveni..."
2,2742092,Bel canto : a novel / Ann Patchett.,"Patchett, Ann","0060838728, 9780060838720",nafic,NA-Fiction,Print,Book,Fiction,Adult,NaN,BOOK,10,"0060838728, 9780060838720","Women singers Fiction, Victims of terrorism Fi..."
3,2305268,Your pal Mo Willems presents Leonardo the terr...,"Willems, Mo",0786852941,ncpic,NC--Children's Picture Books,Print,Book,Fiction,Juvenile,Picture,BOOK,19,0786852941,"Monsters Juvenile fiction, Friendship Fiction,..."
4,2305268,Your pal Mo Willems presents Leonardo the terr...,"Willems, Mo",0786852941,ncpic,NC--Children's Picture Books,Print,Book,Fiction,Juvenile,Picture,BOOK,19,0786852941,"Monsters Juvenile fiction, Friendship Fiction,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,2277368,Harry Potter and the half-blood prince / by J....,"Rowling, J. K.",0439784549,ncfic,NC-Children's Fiction,Print,Book,Fiction,Juvenile,NaN,BOOK,19,0439784549,"Potter Harry Juvenile fiction, Hogwarts School..."
264,2411510,Old man's war / John Scalzi.,"Scalzi, John, 1969-","0765315246, 9780765315243",nasf,NA-Sci-Fic/Fantasy,Print,Book,Fiction,Adult,NaN,BOOK,7,"0765315246, 9780765315243","Life on other planets Fiction, Space colonies ..."
265,3199559,The karate kid / Columbia Pictures ; a Jerry W...,NaN,"1404954333, 140497380X, 9781404954335, 9781404...",nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult,NaN,VIDEODISC,8,"1404954333, 140497380X, 9781404954335, 9781404...","Bullying Drama, Karate California Los Angeles ..."
266,3269894,The big Lebowski / Universal Studios ; Polygra...,NaN,"1417034718, 9781417034710",nadvd,"NA-DVD, Fiction",Media,Video Disc,Fiction,Adult,NaN,VIDEODISC,21,"1417034718, 9781417034710","Slackers Drama, Unemployed Drama, Bowlers Dram..."


### Welche der im Januar 2023 meistausgeliehenen Titel mit Erscheinungsjahr 2005 sind auf Kinder ausgerichtet?

In [562]:
df_checkout_lci_dd_combined[df_checkout_lci_dd_combined["age_group"] == "Juvenile"]

,bibnum,title,author,isbn_x,itemcollection,description,format_group,format_subgroup,category_group,age_group,category_subgroup,materialtype,checkouts,isbn_y,subjects
0,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ccrdr,CC-Children's Readers,Print,Book,Fiction,Juvenile,Picture,BOOK,21,"0375829652, 0375929657","Camping Juvenile fiction, Individuality Juveni..."
1,2310948,Fred and Ted go camping / by Peter Eastman.,"Eastman, Peter, 1942-2020","0375829652, 0375929657",ncrdr,NC--Children's Readers,Print,Book,Fiction,Juvenile,Picture,BOOK,21,"0375829652, 0375929657","Camping Juvenile fiction, Individuality Juveni..."
3,2305268,Your pal Mo Willems presents Leonardo the terr...,"Willems, Mo",0786852941,ncpic,NC--Children's Picture Books,Print,Book,Fiction,Juvenile,Picture,BOOK,19,0786852941,"Monsters Juvenile fiction, Friendship Fiction,..."
4,2305268,Your pal Mo Willems presents Leonardo the terr...,"Willems, Mo",0786852941,ncpic,NC--Children's Picture Books,Print,Book,Fiction,Juvenile,Picture,BOOK,19,0786852941,"Monsters Juvenile fiction, Friendship Fiction,..."
6,2280859,A potty for me! : a lift-the-flap instruction ...,"Katz, Karen",0689874235,ncpic,NC--Children's Picture Books,Print,Book,Fiction,Juvenile,Picture,BOOK,9,0689874235,Toilet training Juvenile fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2277368,Harry Potter and the half-blood prince / by J....,"Rowling, J. K.",0439784549,ncfic,NC-Children's Fiction,Print,Book,Fiction,Juvenile,NaN,BOOK,19,0439784549,"Potter Harry Juvenile fiction, Hogwarts School..."
261,2361194,The wonderful Wizard of Oz / L. Frank Baum ; i...,"Baum, L. Frank (Lyman Frank), 1856-1919",9781402725043,ncfic,NC-Children's Fiction,Print,Book,Fiction,Juvenile,NaN,BOOK,7,9781402725043,Wizard of Oz Fictitious character Juvenile fic...
262,2852359,"Go, train, go! : a Thomas the Tank Engine stor...",NaN,"0375831770, 9780375831775",ncrdr,NC--Children's Readers,Print,Book,Fiction,Juvenile,Picture,BOOK,12,"0375831770, 9780375831775",Thomas the Tank Engine Fictitious character Ju...
263,2277368,Harry Potter and the half-blood prince / by J....,"Rowling, J. K.",0439784549,ncfic,NC-Children's Fiction,Print,Book,Fiction,Juvenile,NaN,BOOK,19,0439784549,"Potter Harry Juvenile fiction, Hogwarts School..."


In [557]:
df_checkout_lci_dd_combined_reduced_title_author = pd.merge(lci_tables_combined, df_physi
                                                            cal, how="inner", left_on=["title","author"], right_on=["title","creator"])

In [556]:
df_checkout_lci_dd_combined_reduced_title_author[pd.isnull(df_checkout_lci_dd_combined_reduced_title_author["isbn_x"])]

,bibnum,title,author,isbn_x,itemcollection,description,format_group,format_subgroup,category_group,age_group,...,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,isbn_y,creator,subjects,publisher,publicationyear


In [365]:
df_checkout_lci_dd_combined_reduced = df_checkout_lci_dd_combined[["title_x", "title_y", "creator", "author"]]

In [366]:
df_checkout_lci_dd_combined_reduced

,title_x,title_y,creator,author
0,Aqualicious / written and illustrated by Victo...,Aqualicious / written and illustrated by Victo...,"Kann, Victoria","Kann, Victoria"
1,"Brown bear, brown bear, what do you see? / by ...","Brown bear, brown bear, what do you see? / by ...","Martin, Bill, 1916-2004","Martin, Bill, 1916-2004"
2,Interrupting chicken / David Ezra Stein.,Interrupting chicken / David Ezra Stein.,"Stein, David Ezra","Stein, David Ezra"
3,Pete the Cat's groovy guide to love : tips fro...,Pete the Cat's groovy guide to love : tips fro...,"Dean, Kim, 1969-","Dean, Kim, 1969-"
4,"Squish. [No. 4], Captain Disaster / by Jennife...","Squish. [No. 4], Captain Disaster / by Jennife...","Holm, Jennifer L.","Holm, Jennifer L."
...,...,...,...,...
184,"Amulet. Book six, Escape from Lucien / Kazu Ki...","Amulet. Book six, Escape from Lucien / Kazu Ki...","Kibuishi, Kazu, 1978-","Kibuishi, Kazu, 1978-"
185,"Slaughterhouse-five, or, The children's crusad...","Slaughterhouse-five, or, The children's crusad...","Vonnegut, Kurt","Vonnegut, Kurt"
186,Stone soup / retold and illustrated by Jon J M...,Stone soup / retold and illustrated by Jon J M...,"Muth, Jon J.","Muth, Jon J."
187,"Squish. [No. 8], Pod vs. pod / by Jennifer L. ...","Squish. [No. 8], Pod vs. pod / by Jennifer L. ...","Holm, Jennifer L.","Holm, Jennifer L."


### Tabelle für Codes von Interlibrary Loans

In [155]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json?category_group='Interlibrary%20Loan'"
local_file = "seattle_il.json"
urllib.request.urlretrieve(url, local_file)

('seattle_il.json', <http.client.HTTPMessage at 0x7983fe948350>)

In [156]:
df_data_dict["category_group"].unique()

array(['Periodical', 'Miscellaneous', 'Fiction', 'Nonfiction',
       'Reference', nan, 'Interlibrary Loan', 'Temporary', 'Analytic',
       'WTBBL', 'Uncataloged', 'Language', 'On Order'], dtype=object)

In [164]:
df_il = pd.read_json("seattle_il.json")
df_il

,code,description,code_type,category_group,format_group,age_group
0,ill,Interlibrary Loan,Location,Interlibrary Loan,NaN,NaN
1,swill,Interlibrary Loan,ItemCollection,Interlibrary Loan,Other,Adult
2,dcillb,Ill Borrowed,ItemType,Interlibrary Loan,Other,Adult
3,dcilll,Ill Lending,ItemType,Interlibrary Loan,Other,Adult


### Tabelle für Codes der Subgroups

In [138]:
df_data_dict["category_subgroup"].unique()

array([nan, 'Biography', 'Peak Picks', 'Comic', 'Large Print', 'Holiday',
       'Picture', 'ESL'], dtype=object)

In [160]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json?$where=category_subgroup%20IS%20NOT%20NULL"
local_file = "seattle_sub.json"
urllib.request.urlretrieve(url, local_file)

('seattle_sub.json', <http.client.HTTPMessage at 0x7983fe84b2c0>)

In [235]:
df_sub = pd.read_json("seattle_sub.json")
df_comics = df_sub[df_sub["category_subgroup"] == "Comic"]

In [236]:
comic_codes = df_comics["code"].unique()

In [237]:
comic_codes

array(['cccomic', 'nacomic', 'cacomic', 'nccomic', 'cycomic', 'nycomic'],
      dtype=object)

In [242]:
for x in comic_codes:
    print(x)

cccomic
nacomic
cacomic
nccomic
cycomic
nycomic


In [238]:
type(comic_codes)

numpy.ndarray

## ISBN

### Daten runterladen

In [19]:
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=(starts_with(isbn,%20'978')%20OR%20starts_with(isbn,'979'))%20AND%20checkoutyear=2023&$limit=10000"
local_file = "checkout.csv"
urllib.request.urlretrieve(url, local_file)

('checkout.csv', <http.client.HTTPMessage at 0x798406dbc6e0>)

In [20]:
df_seattle_isbn = pd.read_csv(local_file)

In [21]:
df_seattle_isbn

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
0,Digital,OverDrive,EBOOK,2023,3,10,Self-Portrait with Nothing,9781250820853,Aimee Pokwatka,"Fiction, Literature, Mystery, Science Fiction",Macmillan Publishers,2022
1,Digital,OverDrive,EBOOK,2023,3,6,The Fervor,9780593328347,Alma Katsu,"Fiction, Horror, Thriller, Historical Fiction","Penguin Group (USA), Inc.",2022
2,Digital,OverDrive,EBOOK,2023,3,1,Heidi Heckelbeck and the Christmas Surprise,9781442481268,Wanda Coven,Juvenile Fiction,"Simon & Schuster, Inc.",2014
3,Digital,OverDrive,EBOOK,2023,3,2,Selected Short Fiction,9780786512362,Charles Dickens,"Classic Literature, Fiction, Short Stories","Penguin Group (USA), Inc.",2011
4,Digital,OverDrive,EBOOK,2023,3,3,Bellweather Rhapsody,9780544133471,Kate Racculia,"Fiction, Literature, Mystery",HarperCollins Publishers Inc.,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Digital,OverDrive,AUDIOBOOK,2023,1,2,The Teacher's Funeral: A Comedy in Three Parts...,9780739345238,Richard Peck,"Historical Fiction, Young Adult Fiction, Young...",Books on Tape,2006
9996,Digital,OverDrive,EBOOK,2023,1,1,We All Fall Down: The River City Duology Serie...,9780374314330,Rose Szabo,"Fantasy, Young Adult Fiction, LGBTQIA+ (Fiction)",Macmillan Publishers,2022
9997,Digital,OverDrive,EBOOK,2023,1,1,The Last Days of Socrates,9780141965888,Plato,"Classic Literature, Fiction",Penguin Random House UK,2010
9998,Digital,OverDrive,EBOOK,2023,1,1,The Children of Henry VIII,9780307806864,Alison Weir,"Biography & Autobiography, History, Nonfiction","Random House, Inc.",2013


In [22]:
df_seattle_isbn.keys()

Index(['usageclass', 'checkouttype', 'materialtype', 'checkoutyear',
       'checkoutmonth', 'checkouts', 'title', 'isbn', 'creator', 'subjects',
       'publisher', 'publicationyear'],
      dtype='object')

In [23]:
df_seattle_isbn["materialtype"].unique()

array(['EBOOK', 'AUDIOBOOK', 'BOOK', 'VIDEODISC', 'SOUNDDISC'],
      dtype=object)

In [24]:
df_seattle_isbn[df_seattle_isbn["materialtype"]=="AUDIOBOOK"]

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
6,Digital,OverDrive,AUDIOBOOK,2023,3,2,Nine Inches: Stories (unabridged),9781427235305,Tom Perrotta,"Fiction, Literature, Short Stories",Macmillan Audio,2013
7,Digital,OverDrive,AUDIOBOOK,2023,3,1,Undiscovered Country (unabridged),9781415956243,Lin Enger,"Fiction, Literature",Books on Tape,2018
12,Digital,OverDrive,AUDIOBOOK,2023,3,1,Just One Touch (unabridged),9780062674326,Maya Banks,"Fiction, Romance, Suspense",HarperCollins Publishers Inc.,2017
17,Digital,OverDrive,AUDIOBOOK,2023,3,2,First Born: A Novel (unabridged),9781797142814,Will Dean,"Fiction, Suspense, Thriller",Simon & Schuster - Audiobooks,2022
18,Digital,OverDrive,AUDIOBOOK,2023,3,5,Berlin Alexanderplatz (unabridged),9781977353825,Alfred Doblin,"Fiction, Literature","Tantor Media, Inc",2020
...,...,...,...,...,...,...,...,...,...,...,...,...
9987,Digital,OverDrive,AUDIOBOOK,2023,1,2,The Wires of War: Technology and the Global St...,9781797132723,Jacob Helberg,"Computer Technology, Politics, Nonfiction",Simon & Schuster - Audiobooks,2021
9988,Digital,OverDrive,AUDIOBOOK,2023,1,9,Winter's Orbit (unabridged),9781250790194,Everina Maxwell,"Fiction, Science Fiction",Macmillan Audio,2021
9990,Digital,OverDrive,AUDIOBOOK,2023,1,8,"They Do It With Mirrors: Miss Marple Series, B...",9781408484890,Agatha Christie,"Fiction, Mystery, Thriller",Penguin Random House UK,2021
9992,Digital,OverDrive,AUDIOBOOK,2023,1,45,Portrait of a Thief: A Novel (unabridged),9780593510964,Grace D. Li,"Fiction, Literature, Mystery",Books on Tape,2022


In [25]:
df_seattle_isbn[df_seattle_isbn["materialtype"]=="VIDEODISC"]

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
4391,Physical,Horizon,VIDEODISC,2023,1,2,Finding your roots. Season 2 / with Henry Loui...,9781627890946,NaN,"Genealogy, Celebrities Genealogy, Celebrities ...","Distributed by PBS Distribution,",[2014]
4496,Physical,Horizon,VIDEODISC,2023,1,1,Craft in America. Celebration / executive prod...,9781627894258,NaN,"Handicraft United States History 20th century,...","Distributed by PBS Distribution,",c2016.
7102,Physical,Horizon,VIDEODISC,2023,1,5,The fugitive kind / 20th Century Fox ; Metro-G...,9781604652703,NaN,"Triangles Interpersonal relations Drama, Drift...","Criterion Collection,",2010.
8999,Physical,Horizon,VIDEODISC,2023,1,2,Defiant Requiem / a Partisan Pictures producti...,9781608839179,NaN,"Sch„chter Raphael, Verdi Giuseppe 1813 1901 Me...","PBS Distribution,",[2013]


In [26]:
df_seattle_isbn["usageclass"].value_counts()

usageclass
Digital     9942
Physical      58
Name: count, dtype: int64

### Download physical with 978 ISBN

In [27]:
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=(starts_with(isbn,%20'978')%20OR%20starts_with(isbn,'979'))%20AND%20usageclass='Physical'&$limit=10000"
local_file = "checkout2.csv"
urllib.request.urlretrieve(url, local_file)

('checkout2.csv', <http.client.HTTPMessage at 0x798406be2180>)

In [28]:
df_physical = pd.read_csv(local_file)

In [29]:
df_physical

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
0,Physical,Horizon,BOOK,2022,5,17,Macrina Bakery : seasons / written by Leslie M...,"9780578614939, 9789910387715, 9910387719","Mackie, Leslie","Seasonal cooking, Macrina Bakery and Cafe, Foo...","Macrina Bakery,",[2019]
1,Physical,Horizon,BOOK,2022,5,1,The red apple / Feridun Oral.,"9789888240005, 9888240005","Oral, Feridun","Friendship Juvenile fiction, Cooperation Juven...","Minedition (Michael Neugebauer Publishing Ltd.),",[2015]
2,Physical,Horizon,BOOK,2022,5,1,Tai cai ben wei : Huang Wanling de cai wei tan...,"9789869899659, 986989965X","Huang, Wanling","Cooking Chinese Taiwan style, Snack foods Taiw...","Xie le wen hua you xian gong si,",2021.
3,Physical,Horizon,BOOK,2022,5,1,Mr. Hollyberry's Christmas gift / by Kate West...,"9789888341627, 9888341626","Westerlund, Kate","Snowmen Juvenile fiction, Cats Juvenile fictio...","Michael Neugebauer Publishing,",2018.
4,Physical,Horizon,BOOK,2022,5,3,Geek love / Katherine Dunn.,9780375713347,"Dunn, Katherine, 1945-2016","Carnival owners Fiction, Circus performers Fic...","Vintage Books,",2002.
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Physical,Horizon,BOOK,2023,4,1,Adaku & other stories / Ken Saro-Wiwa.,9782460117,"Saro-Wiwa, Ken, 1941-1995",NaN,"Saros International Publishers,",1989.
9996,Physical,Horizon,BOOK,2023,4,5,The forgotten crayon / Yoko Maruyama.,"9789888341986, 9888341987","Maruyama, Yoko (Children's author)","Crayons Juvenile fiction, Picture books","Michael Neugebauer Publishing,",2019.
9997,Physical,Horizon,BOOK,2023,4,3,Korean children's favorite stories / retold by...,9780804835916,"Kim, So-un, 1907-1981","Tales Korea Juvenile literature, Folklore Kore...","Tuttle Pub.,",[2004]
9998,Physical,Horizon,VIDEODISC,2023,4,31,Bergman island / IFC Films and CG Cinéma prese...,"9789798886072, 9798886070, 9798886070125",NaN,"Bergman Ingmar 1918 2007 Drama, Independent fi...","Criterion Collection,",[2023]
